In [361]:
#importing modules
import pandas as pd
from pathlib import Path

In [362]:
#setting file path to read
voc_path = Path("Resources/voc_rehab.csv")
unemployment_path = Path("Resources/unemployment.csv")
success_path = Path('Resources/successful_closures.csv')

# Read with Pandas
vocation_df = pd.read_csv(voc_path)
vocation_df.head()

,Year,County,Sex,Number of Consumers,Annotation Code,Count Annotation Desc,Percent,Percent Annotation Desc
0,2014,Alameda,Not Reported,4,NaN,NaN,NaN,NaN
1,2014,Alameda,Female,"2,356",NaN,NaN,NaN,NaN
2,2014,Alameda,Male,"2,391",NaN,NaN,NaN,NaN
3,2014,Alpine,Female,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
4,2014,Amador,Female,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...


In [363]:
#shows what columns to determine relevance for vocational data
vocation_df.columns

Index(['Year', 'County', 'Sex', 'Number of Consumers', 'Annotation Code',
       'Count Annotation Desc', 'Percent', 'Percent Annotation Desc'],
      dtype='object')

In [364]:
#removing irrelevant columns
del vocation_df["Annotation Code"]
del vocation_df["Count Annotation Desc"]
del vocation_df["Percent"]
del vocation_df["Percent Annotation Desc"]
del vocation_df["Sex"]

#removing commas from numerical data
vocation_df = vocation_df.replace(',', '', regex=True)
vocation_df.head()

,Year,County,Number of Consumers
0,2014,Alameda,4
1,2014,Alameda,2356
2,2014,Alameda,2391
3,2014,Alpine,NaN
4,2014,Amador,NaN


In [365]:
#filling na values w/0
vocation_df = vocation_df.fillna(0)

#filtering data by year
vocation_df = vocation_df.set_index('Year')

#changing 'number of consumers' to type int
vocation_df = vocation_df.astype({'Number of Consumers': int})

vocation_df.head()

,County,Number of Consumers
Year,,
2014,Alameda,4
2014,Alameda,2356
2014,Alameda,2391
2014,Alpine,0
2014,Amador,0


In [367]:
#adding together total number of consumers per coutny per year
vocation_df = vocation_df.groupby(['Year', 'County'])['Number of Consumers'].sum()
vocation_df

Year  County   
2014  Alameda      4751
      Alpine          0
      Amador         18
      Butte         735
      Calaveras      30
                   ... 
2017  Tulare       1117
      Tuolumne       87
      Ventura      2878
      Yolo          499
      Yuba          211
Name: Number of Consumers, Length: 240, dtype: int64

In [355]:
#introducing unemployment data
unemployment_df = pd.read_csv(unemployment_path)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
unemployment_df = pd.read_csv(unemployment_path)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
area_type_df = unemployment_df.loc[['State','County'],
                                   ['Area Name', 'Year', 'Labor Force', 'Employment', 'Unemployment', 'Unemployment Rate']]
area_type_df

# Group by Year and calculate averages
year_grouped = area_type_df.groupby(['Year', 'Area Name']).mean()
year_grouped

# Reduce to only 2014-2017 to line up with vocational data
recent = year_grouped.loc[[2014, 2015, 2016, 2017]]
recent

Labor Force     Employment  Unemployment  \
Year Area Name                                                      
2014 Alameda County    806325.000000  758808.333333  47500.000000   
     Alpine County        509.166667     466.666667     41.666667   
     Amador County      14322.500000   13118.333333   1204.166667   
     Butte County      101058.333333   92208.333333   8841.666667   
     Calaveras County   20204.166667   18569.166667   1635.000000   
...                              ...            ...           ...   
2017 Tulare County     204458.333333  182966.666667  21491.666667   
     Tuolumne County    21369.166667   20207.500000   1160.833333   
     Ventura County    423675.000000  404475.000000  19225.000000   
     Yolo County       106558.333333  101091.666667   5483.333333   
     Yuba County        28716.666667   26566.666667   2141.666667   

                       Unemployment Rate  
Year Area Name                            
2014 Alameda County             0.059000  
     Alpine County              0.083167  
     Amador County              0.083917  
     Butte County               0.087500  
     Calaveras County           0.081083  
...                                  ...  
2017 Tulare County              0.105167  
     Tuolumne County            0.054250  
     Ventura County             0.045333  
     Yolo County                0.051500  
     Yuba County                0.074667  

[236 rows x 4 columns]

In [356]:
#introducing successful vocational services
success_df = pd.read_csv(success_path)
success_df

,Year,County,Occupation,Successful Closures,Annotation Code,Count Annotation Desc,Percent,Percent Annotation Desc
0,2014,Alameda,Not Reported,67.0,NaN,NaN,NaN,NaN
1,2014,Alameda,Clerical and Administrative Support,114.0,NaN,NaN,NaN,NaN
2,2014,Alameda,Community and Social Service Occupations,3.0,NaN,NaN,NaN,NaN
3,2014,Alameda,Computer and Mathematical Occupations,5.0,NaN,NaN,NaN,NaN
4,2014,Alameda,"Education, Training, and Library Occupations",4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1935,2017,Yuba,"Professional, Paraprofessional and Technical",NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
1936,2017,Yuba,Protective Service Occupations,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
1937,2017,Yuba,RSA Special Occupations and Miscellaneous,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
1938,2017,Yuba,Sales and Related Occupations,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...


In [357]:
#displaying columns to display relevance
success_df.columns

Index(['Year', 'County', 'Occupation', 'Successful Closures',
       'Annotation Code', 'Count Annotation Desc', 'Percent',
       'Percent Annotation Desc'],
      dtype='object')

In [358]:
#removing irrelevant columns
del success_df['Occupation']
del success_df['Annotation Code']
del success_df['Count Annotation Desc']
del success_df['Percent']
del success_df['Percent Annotation Desc']

#replacing na w/ value of 0
success_df = success_df.fillna(0)
success_df

,Year,County,Successful Closures
0,2014,Alameda,67.0
1,2014,Alameda,114.0
2,2014,Alameda,3.0
3,2014,Alameda,5.0
4,2014,Alameda,4.0
...,...,...,...
1935,2017,Yuba,0.0
1936,2017,Yuba,0.0
1937,2017,Yuba,0.0
1938,2017,Yuba,0.0


In [359]:
#adding total number of successful closures per county per year
success_df = success_df.groupby(['Year', 'County'])['Successful Closures'].sum()
success_df

Year  County   
2014  Alameda      554.0
      Amador         0.0
      Butte        113.0
      Calaveras      1.0
      Colusa         0.0
                   ...  
2017  Tulare        75.0
      Tuolumne       0.0
      Ventura      301.0
      Yolo          50.0
      Yuba           1.0
Name: Successful Closures, Length: 230, dtype: float64